### Manipulation of PeaskDB de novo-assisted database search results of Trocas 8 (low water, November 2019) non incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 683) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/683_Trocas8_June2021_PEAKS_142/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/683_Trocas8_June2021_PEAKS_142


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 683_Trocas8_June2021_PEAKS_142_peptide.csv     'de novo only peptides.csv'
 683_Trocas8_June2021_PEAKS_142_proteins.csv     peptides_1_1_0.mzid
 683_Trocas8_June2021_PEAKS_142_proteins.fasta   protein-peptides.csv
'DB search psm.csv'


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup683 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/683_Trocas8_June2021_PEAKS_142/683_Trocas8_June2021_PEAKS_142_peptide.csv")

# remove redundant rows
peaksdb683 = pd.DataFrame.drop_duplicates(peaksdbdup683)

print(peaksdb683.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb683.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb683['#Spec']
del peaksdb683['Accession']
del peaksdb683['PTM']
del peaksdb683['AScore']

mean_length = peaksdb683['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup683))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb683))

#look at the dataframe
peaksdb683.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Sample 9', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Sample 9', 'Accession', 'PTM', 'AScore'],
      dtype='object')
9.475
# redundant peaksdb peptides in combined dataframe 40
# nonredundant peaksdb peptides in combined dataframe 40


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LSSPATLNSR,44.68,1044.5564,10,-0.4,523.2853,44.47,3940000.0,9,6761,20210624_Trocas8_683_Baylique55_50%_DDA_120min...
1,EGNEQFINAAK,39.92,1219.5833,11,-1.5,610.7980,51.03,54100.0,9,7843,20210624_Trocas8_683_Baylique55_50%_DDA_120min...
2,LATVLSPR,37.57,855.5178,8,-1.6,428.7655,58.47,1700000.0,9,8939,20210624_Trocas8_683_Baylique55_50%_DDA_120min...
3,LATVISPR,37.57,855.5178,8,-1.6,428.7655,58.47,1700000.0,9,8939,20210624_Trocas8_683_Baylique55_50%_DDA_120min...
4,YYTLEEIQK,35.84,1185.5917,9,-1.1,593.8025,72.83,29600.0,9,11002,20210624_Trocas8_683_Baylique55_50%_DDA_120min...


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb683['A'] = peaksdb683['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb683['C'] = peaksdb683['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb683['D'] = peaksdb683['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb683['E'] = peaksdb683['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb683['F'] = peaksdb683['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb683['G'] = peaksdb683['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb683['H'] = peaksdb683['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb683 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb683['I'] = peaksdb683['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb683['K'] = peaksdb683['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb683['L'] = peaksdb683['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb683['M'] = peaksdb683['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb683['N'] = peaksdb683['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb683['P'] = peaksdb683['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb683['Q'] = peaksdb683['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb683['R'] = peaksdb683['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb683['S'] = peaksdb683['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb683['T'] = peaksdb683['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb683['V'] = peaksdb683['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb683['W'] = peaksdb683['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb683['Y'] = peaksdb683['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb683['c-carb'] = peaksdb683['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb683['m-oxid'] = peaksdb683['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb683['n-deam'] = peaksdb683['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb683['q-deam'] = peaksdb683['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb683['stripped_peptide'] = peaksdb683['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb683['stripped_length'] = peaksdb683['stripped_peptide'].apply(len)

peaksdb683['NAAF_num.'] = peaksdb683['Area'] / peaksdb683['stripped_length']

# total the number of modifications in sequence
peaksdb683['ptm-total'] = peaksdb683['c-carb'] + peaksdb683['m-oxid'] + peaksdb683['n-deam'] + peaksdb683['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb683['stripped_IL']= peaksdb683['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/675_BY_50_GF75_PDB/683_BY_50_GF75_PDB.csv")

# check out the results
peaksdb683.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LSSPATLNSR,44.68,1044.5564,10,-0.4,523.2853,44.47,3940000.0,9,6761,...,0,0,0,0,0,LSSPATLNSR,10,394000.000000,0,LSSPATLNSR
1,EGNEQFINAAK,39.92,1219.5833,11,-1.5,610.7980,51.03,54100.0,9,7843,...,0,0,0,0,0,EGNEQFINAAK,11,4918.181818,0,EGNEQFLNAAK
2,LATVLSPR,37.57,855.5178,8,-1.6,428.7655,58.47,1700000.0,9,8939,...,0,0,0,0,0,LATVLSPR,8,212500.000000,0,LATVLSPR
3,LATVISPR,37.57,855.5178,8,-1.6,428.7655,58.47,1700000.0,9,8939,...,0,0,0,0,0,LATVISPR,8,212500.000000,0,LATVLSPR
4,YYTLEEIQK,35.84,1185.5917,9,-1.1,593.8025,72.83,29600.0,9,11002,...,2,0,0,0,0,YYTLEEIQK,9,3288.888889,0,YYTLEELQK


In [6]:
# keep only stripped peptide I/L and NAAF
db_683 = peaksdb683[['stripped_IL', 'Area', 'NAAF_num.']]

db_683.set_index('stripped_IL')

# write modified dataframe to new txt file
db_683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/683_BY_50_GF75_PDB_ILnaafs.csv")

db_683.head()

,stripped_IL,Area,NAAF_num.
0,LSSPATLNSR,3940000.0,394000.000000
1,EGNEQFLNAAK,54100.0,4918.181818
2,LATVLSPR,1700000.0,212500.000000
3,LATVLSPR,1700000.0,212500.000000
4,YYTLEELQK,29600.0,3288.888889


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep683moddup = peaksdb683[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep683dup = peaksdb683[["stripped_peptide"]]

# deduplicate both of these lists
pep683mod = pep683moddup.drop_duplicates()
pep683 = pep683dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 683:", len(pep683moddup))
print("Deduplicated modified PeaksDB peptides in 683:", len(pep683mod))
print("Total stripped PeaksDB peptides in 683:", len(pep683dup))
print("Deduplicated stripped PeaksDB peptides in 683:", len(pep683))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/675_BY_50_GF75_PDB/683_BY_50_GF75_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/683_BY_50_GF75_PDB/683_BY_50_GF75_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/683_BY_50_GF75_PDB/683_BY_50_GF75_PDB_stripped_peptides.fas

# look at the stripped peptides
pep683.head()

Total modified PeaksDB peptides in 683: 40
Deduplicated modified PeaksDB peptides in 683: 40
Total stripped PeaksDB peptides in 683: 40
Deduplicated stripped PeaksDB peptides in 683: 39
/bin/bash: line 1: /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/683_BY_50_GF75_PDB/683_BY_50_GF75_PDB_stripped_peptides.fas: No such file or directory


,stripped_peptide
0,LSSPATLNSR
1,EGNEQFINAAK
2,LATVLSPR
3,LATVISPR
4,YYTLEEIQK


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb683['A'].sum(),
        'C': peaksdb683['C'].sum(),
        'D': peaksdb683['D'].sum(),
        'E': peaksdb683['E'].sum(),
        'F': peaksdb683['F'].sum(),
        'G': peaksdb683['G'].sum(),
        'H': peaksdb683['H'].sum(),
        'I': peaksdb683['I'].sum(),
        'K': peaksdb683['K'].sum(),
        'L': peaksdb683['L'].sum(),
        'M': peaksdb683['M'].sum(),
        'N': peaksdb683['N'].sum(),
        'P': peaksdb683['P'].sum(),
        'Q': peaksdb683['Q'].sum(),
        'R': peaksdb683['R'].sum(),
        'S': peaksdb683['S'].sum(),
        'T': peaksdb683['T'].sum(),
        'V': peaksdb683['V'].sum(),
        'W': peaksdb683['W'].sum(),
        'Y': peaksdb683['Y'].sum(),
        'c-carb': peaksdb683['c-carb'].sum(),
        'm-oxid': peaksdb683['m-oxid'].sum(),
        'n-deam': peaksdb683['n-deam'].sum(),
        'q-deam': peaksdb683['q-deam'].sum(),
        'Total area': peaksdb683['Area'].sum(),
        'Total length': peaksdb683['stripped_length'].sum()
       }

totalpeaksdb683 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb683['% C w/ carb'] = totalpeaksdb683['c-carb'] / totalpeaksdb683['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb683['% M w/ oxid'] = totalpeaksdb683['m-oxid'] / totalpeaksdb683['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb683['% N w/ deam'] = totalpeaksdb683['n-deam'] / totalpeaksdb683['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb683['% Q w/ deam'] = totalpeaksdb683['q-deam'] / totalpeaksdb683['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb683['NAAF denom.'] = totalpeaksdb683['Total area'] / totalpeaksdb683['Total length']

# write modified dataframe to new txt file
totalpeaksdb683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/675_BY_50_GF75_PDB/683_BY_50_GF75_PDB_totals.csv")

totalpeaksdb683.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,35,2,9,16,11,18,10,25,27,50,...,0,2,2,18056150.0,370,1.0,0.0,0.166667,0.153846,48800.405405


In [9]:
# use the calculated NAAF factor (in totalpeaksdb683 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 48800.405405

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb683['NAAF factor'] = (peaksdb683['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb683_NAAF = peaksdb683[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb683_NAAF['A-NAAF20'] = peaksdb683_NAAF['A'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['C-NAAF20'] = peaksdb683_NAAF['C'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['D-NAAF20'] = peaksdb683_NAAF['D'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['E-NAAF20'] = peaksdb683_NAAF['E'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['F-NAAF20'] = peaksdb683_NAAF['F'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['G-NAAF20'] = peaksdb683_NAAF['G'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['H-NAAF20'] = peaksdb683_NAAF['H'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['I-NAAF20'] = peaksdb683_NAAF['I'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['K-NAAF20'] = peaksdb683_NAAF['K'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['L-NAAF20'] = peaksdb683_NAAF['L'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['M-NAAF20'] = peaksdb683_NAAF['M'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['N-NAAF20'] = peaksdb683_NAAF['N'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['P-NAAF20'] = peaksdb683_NAAF['P'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['Q-NAAF20'] = peaksdb683_NAAF['Q'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['R-NAAF20'] = peaksdb683_NAAF['R'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['S-NAAF20'] = peaksdb683_NAAF['S'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['T-NAAF20'] = peaksdb683_NAAF['T'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['V-NAAF20'] = peaksdb683_NAAF['V'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['W-NAAF20'] = peaksdb683_NAAF['W'] * peaksdb683['NAAF factor']
peaksdb683_NAAF['Y-NAAF20'] = peaksdb683_NAAF['Y'] * peaksdb683['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb683_NAAF['ccarb-NAAF20'] = peaksdb683_NAAF['c-carb'] * peaksdb683_NAAF['NAAF factor']
peaksdb683_NAAF['moxid-NAAF20'] = peaksdb683_NAAF['m-oxid'] * peaksdb683_NAAF['NAAF factor']
peaksdb683_NAAF['ndeam-NAAF20'] = peaksdb683_NAAF['n-deam'] * peaksdb683_NAAF['NAAF factor']
peaksdb683_NAAF['qdeam-NAAF20'] = peaksdb683_NAAF['q-deam'] * peaksdb683_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb683_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/675_BY_50_GF75_PDB/683_BY_50_GF75_PDB_naaf.csv")

peaksdb683_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,LSSPATLNSR,8.073703,1,0,0,0,0,0,0,0,...,8.073703,24.221110,8.073703,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,EGNEQFINAAK,0.100782,2,0,0,2,1,1,0,1,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,LATVLSPR,4.354472,1,0,0,0,0,0,0,0,...,4.354472,4.354472,4.354472,4.354472,0.0,0.000000,0.0,0.0,0.0,0.0
3,LATVISPR,4.354472,1,0,0,0,0,0,0,1,...,4.354472,4.354472,4.354472,4.354472,0.0,0.000000,0.0,0.0,0.0,0.0
4,YYTLEEIQK,0.067395,0,0,0,2,0,0,0,1,...,0.000000,0.000000,0.067395,0.000000,0.0,0.134789,0.0,0.0,0.0,0.0


In [10]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb683 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb683_NAAF['NAAF factor'].sum(),
        'A': peaksdb683_NAAF['A-NAAF20'].sum(),
        'C': peaksdb683_NAAF['C-NAAF20'].sum(),
        'D': peaksdb683_NAAF['D-NAAF20'].sum(),
        'E': peaksdb683_NAAF['E-NAAF20'].sum(),
        'F': peaksdb683_NAAF['F-NAAF20'].sum(),
        'G': peaksdb683_NAAF['G-NAAF20'].sum(),
        'H': peaksdb683_NAAF['H-NAAF20'].sum(),
        'I': peaksdb683_NAAF['I-NAAF20'].sum(),
        'K': peaksdb683_NAAF['K-NAAF20'].sum(),
        'L': peaksdb683_NAAF['L-NAAF20'].sum(),
        'M': peaksdb683_NAAF['M-NAAF20'].sum(),
        'N': peaksdb683_NAAF['N-NAAF20'].sum(),
        'P': peaksdb683_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb683_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb683_NAAF['R-NAAF20'].sum(),
        'S': peaksdb683_NAAF['S-NAAF20'].sum(),
        'T': peaksdb683_NAAF['T-NAAF20'].sum(),
        'V': peaksdb683_NAAF['V-NAAF20'].sum(),
        'W': peaksdb683_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb683_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb683_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb683_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb683_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb683_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb683_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb683_NAAF['% C w/ carb.'] = totalpeaksdb683_NAAF['c-carb'] / totalpeaksdb683_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb683_NAAF['% M w/ oxid'] = totalpeaksdb683_NAAF['m-oxid'] / totalpeaksdb683_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb683_NAAF['% N w/ deam'] = totalpeaksdb683_NAAF['n-deam'] / totalpeaksdb683_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb683_NAAF['% Q w/ deam'] = totalpeaksdb683_NAAF['q-deam'] / totalpeaksdb683_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb683_NAAF['NAAF check'] = totalpeaksdb683_NAAF['NAAF'] / 48800.405405

# write modified dataframe to new txt file, same name + totals
totalpeaksdb683_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/675_BY_50_GF75_PDB/683_BY_50_GF75_PDB_naaf_totals.csv")

totalpeaksdb683_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,44.087341,30.303411,0.114753,0.676549,0.792367,0.325706,2.612745,0.775408,10.323749,1.824201,...,1.388671,0.114753,0.0,0.080942,0.007975,1.0,NaN,0.009315,0.010078,0.000903
